In [224]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains#滑鼠套件
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import os
import time 
import xl2dict
import smtplib
from email.message import EmailMessage


In [225]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("start-maximized")
# chrome_options.add_argument("disable-infobars")
# chrome_options.add_argument("--disable-extensions")

#crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔 
# chrome_options.add_extension(r'D:\xxx\extension_0_0_1_3.crx')
#webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
#套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon
driver =webdriver.Chrome(executable_path=r'D:\Driver\chromedriver.exe', chrome_options=chrome_options)
driver.get("https://waste.epa.gov.tw/prog/IndexFrame.asp?Func=2")
# driver.s
driver.set_window_size(1050,840)

driver.save_screenshot(r"D:\trydata\screenshot.png")
driver.switch_to_default_content()
driver.switch_to_frame(1)
driver.page_source
accountInput=driver.find_element_by_name("fac_user")
accountInput.send_keys("A45B1625")

pwInput=driver.find_element_by_name("fac_pwd")
pwInput.send_keys("a!0227581123")

picture=driver.find_element_by_id("CAPTCHA")
# location=picture.location
location={'x':picture.location['x']-395,'y':picture.location['y']+206}
# location={'x':577,"y":326}
# location
left=location['x']
right=location['x']+picture.size['width']
top=location['y']
botton=location['y']+picture.size['height']
from PIL import Image
img=Image.open(r"D:\trydata\screenshot.png")
img=img.crop((left,top,right,botton))
img.save(r"D:\trydata\captcha.png")

# 大神的截圖方法

In [5]:
import os
import base64, re
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
import time 
import xl2dict
import smtplib
from email.message import EmailMessage

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
import cv2
import PIL
import numpy
from matplotlib import pyplot as plt

chrome_options = Options()
def base64_to_image(base64_str, image_path=None):
    base64_data = re.sub('^data:image/.+;base64,', '', base64_str)
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    img = Image.open(image_data)
    if image_path:
        img.save(image_path)

def save_captcha():
    driver.switch_to.default_content()
    driver.switch_to.frame(1)
    try:
        driver.execute_script('''
        img = document.getElementById("CAPTCHA");
        var imgCanvas = document.createElement("canvas"),
        imgContext = imgCanvas.getContext("2d");
        imgCanvas.width = img.width;
        imgCanvas.height = img.height;
        imgContext.drawImage(img, 0, 0, imgCanvas.width, imgCanvas.height);
        imgInfom = imgCanvas.toDataURL("image/png");
        localStorage.setItem("imgInfo",imgInfom);
        ''')
        img_data = driver.execute_script('return localStorage.getItem("imgInfo")')
        base64_to_image(img_data,"./captchaDir/CAPTCHA.png")
    except Exception as e:
        print(str(e))

        
if not os.path.exists("captchaDir"):
    os.mkdir("captchaDir")
driver_path = os.path.join(os.getcwd(), 'chromedriver')
driver = webdriver.Chrome(executable_path=driver_path, chrome_options=chrome_options)
driver.get("https://waste.epa.gov.tw/prog/IndexFrame.asp?Func=2")





success=0
while success==0:
    driver.switch_to.default_content()
    driver.switch_to.frame(1)
    driver.page_source
    accountInput=driver.find_element_by_name("fac_user")
    accountInput.clear()
    accountInput.send_keys("A45B1625")
    pwInput=driver.find_element_by_name("fac_pwd")
    pwInput.clear()
    pwInput.send_keys("a!0227581123")

    ##########################


    captchaText=""
    while len(captchaText)!=4:
        save_captcha()
        mlp=joblib.load(r'D:\trydata\textConsider\model\captcha.pkl')
        # img=Image.open("./captchaDir/CAPTCHA.png")

        pil_image =PIL.Image.open("./captchaDir/CAPTCHA.png").convert("RGB")
        open_cv_image=numpy.array(pil_image)
        plt.imshow(open_cv_image)
        imgray=cv2.cvtColor(open_cv_image,cv2.COLOR_BGR2GRAY)
        ret, thresh=cv2.threshold(imgray,127,255,0)
        image,contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnts=sorted([(c,cv2.boundingRect(c)[0]) for c in contours],key=lambda x:x[1])
        ary=[]
        for (c,_) in cnts:
            (x,y,w,h)=cv2.boundingRect(c)
        #         print((x,y,w,h))
            if w>11 and h>=7 and w!=120:
                if x-2>0:
                    ary.append((x-2,y-2,w+4,h+4))
                else:
                    ary.append((x,y,w+4,h+4))
        n=0

        if not os.path.exists("./captchaDir/numPic/"):
            os.mkdir("./captchaDir/numPic/")

        for i in os.listdir("./captchaDir/numPic/"):
            os.remove("./captchaDir/numPic/"+i)

        for (x,y,w,h) in ary:
            fig=plt.figure(figsize=(6, 4), dpi=100) #600*400
            roi=open_cv_image[y:y+h,x:x+w]
            thresh=roi.copy()
        #     ax=fig.add
            plt.imshow(thresh)
            plt.savefig("./captchaDir/numPic/aa_{}.png".format(n),dpi=100)
            n+=1


        data=[]
        basewidth=50
        fig=plt.figure(figsize=(20,20))
        cnt=0
        for idx,img in enumerate(os.listdir("./captchaDir/numPic/")):
            pil_image=PIL.Image.open("./captchaDir/numPic/{}".format(img)).convert('1')
            wpercent=(basewidth/float(pil_image.size[0]))
            hsize=int((float(pil_image.size[1])*float(wpercent)))
            img=pil_image.resize((basewidth,hsize),PIL.Image.ANTIALIAS)
            data.append([pixel for pixel in iter(img.getdata())])
        scaler=StandardScaler()
        scaler.fit(data)
        data_scaled=scaler.transform(data)
        print(mlp.predict(data_scaled))

        captchaText="".join(mlp.predict(data_scaled)).replace("xx","")
        if len(captchaText)!=4:
            driver.find_element_by_link_text("重新載入圖片").click()
            time.sleep(0.5)


    print(captchaText)
    captchaInput=driver.find_element_by_id("securityCode")
    captchaInput.send_keys(captchaText)
    driver.find_element_by_name("Action").click()
    time.sleep(0.5)
    for i in range(5):
        try:
            alert=driver.switch_to_alert()
            alert.accept()
            time.sleep(0.1)
        except:
            pass

    try:
        time.sleep(0.5)
        driver.find_element_by_link_text("越境聯單資料錯誤說明")
        success=1
    except:
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxx")
        time.sleep(0.5)

600.0
600.0
600.0
600.0
600.0
['xx' '7' 'D' '5' 'E']
7D5E


# 廚餘

In [69]:
driver.get("https://waste.epa.gov.tw/massbal/restaurant.asp")

In [70]:
driver.find_element_by_name("app_year").click()
import datetime
now=datetime.datetime.now()
yearTaiwan=str(now.year-1911)
driver.find_element_by_xpath('//option[@value="' + yearTaiwan + '"]').click()
driver.find_element_by_name("app_year").click()

In [71]:
driver.find_element_by_name("app_month").click()
import datetime
now=datetime.datetime.now()
monthTaiwan=str(now.month-1)
if len(monthTaiwan)==1:
    monthTaiwan="0"+monthTaiwan
driver.find_element_by_xpath('//option[@value="' + monthTaiwan + '"]').click()
driver.find_element_by_name("app_month").click()

In [72]:
driver.find_elements_by_name("submit")[0].click()

In [73]:
xxxx=100 #廚餘
if xxxx==0:
    driver.find_element_by_xpath("/html/body/p[5]/b[2]/font/a").click()
    alert=driver.switch_to_alert()
    alert.accept()
    time.sleep(0.1)
else:
    driver.find_element_by_xpath("/html/body/p[4]/b[2]/font/a").click()
    time.sleep(0.5)
    productInput=driver.find_element_by_id("mod_app_qty")
    productInput.clear()
    productInput.send_keys(str(xxxx))
    includeWaterInput=driver.find_element_by_id("mod_inwater")
    includeWaterInput.clear()
    includeWaterInput.send_keys("0")
    # driver.find_element_by_name("B3").click()
#     alert=driver.switch_to_alert()
#     alert.accept()
#     time.sleep(0.1)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)
